In [1]:
#downloading missing packages
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re

import pickle

import warnings
warnings.filterwarnings('ignore')

### nltk prep

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
nltk.data.path.append('/root/nltk_data/corpora/')

In [6]:
stop_words = stopwords.words('english')
print(stop_words) # some words I like to remove are not included

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
new_words = ['said','like','year','would','house','also','sends']
stop_words.extend(new_words)
stop_words = set(stop_words)

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Dataframe prep

In [9]:
#importing data
file = 'https://raw.githubusercontent.com/Shacham-R/fake_news_detector/main/data/main_df_51k'
df = pd.read_csv(file)

In [10]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [11]:
df.head()

,title_text,fake
0,busted clinton-lynch tarmac meeting documents…...,1
1,democrat senator make fake anti-trump story in...,1
2,congressman tell trump deep wiretapping conspi...,1
3,watch msnbc objective host loud outburst latin...,1
4,u.s. lawmaker reach agreement north korea sanc...,0


### word count calculation

In [12]:
# make a new columns 'token_text' from the text column
df['title_text'] = df['title_text'].astype(str)
df['token_text'] = df['title_text'].apply(lambda x: nltk.word_tokenize(x))

In [13]:
# Count all words in the data
list_words = []
for i in df.token_text:
    for w in i:
        list_words.append(w)

In [14]:
num_words = len(list(set(list_words)))
num_words

248903

In [15]:
del df['token_text']

In [16]:
# Create function to automatically lemmatization and remove stopwords
def lemmatization_and_stopwords(text):
    clean_text = []
    # Set all text into lowercase to match the stopwords
    text = text.lower()
    # Tokenize the text before processing
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()

    for token in tokens:
        if token not in stop_words and len(token)>3:
            token = lemmatizer.lemmatize(token)
            clean_text.append(token)

    text = " ".join(clean_text)

    return text

In [17]:
df['title_text'] = df['title_text'].apply(lemmatization_and_stopwords)

### X, y split

In [18]:
# Split the data into feature and label
X = df['title_text'].values
y = df['fake'].values

# Split the data into train and validation data using train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True)

In [19]:
# Create a tokenizer variable
tokenizer = Tokenizer(num_words=num_words, oov_token='*')


# Apply tokenizer to X_train and X_valid
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_valid)

In [20]:
with open("tokenizer.pickle","wb") as f:
    pickle.dump(tokenizer, f)

In [21]:
# Create text sequences
sequence_train = tokenizer.texts_to_sequences(X_train)
sequence_valid = tokenizer.texts_to_sequences(X_valid)

# Create text padding on post
padding_train = pad_sequences(sequence_train,
                              maxlen=100,
                              padding='post',
                              truncating='post'
                             )

padding_valid = pad_sequences(sequence_valid,
                              maxlen=100,
                              padding='post',
                              truncating='post'
                             )

# SNN Model

In [22]:
# Create sequential model using Embedding, LSTM,
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=num_words, output_dim=128),
    tf.keras.layers.LSTM(256),#256
    tf.keras.layers.Dense(128, activation='relu'),#128
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model using the suitable parameter
model.compile(optimizer='adam',
              metrics=['accuracy'],
              loss='binary_crossentropy'
             )

# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         31859584  
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 32286849 (123.16 MB)
Trainable params: 32286849 (123.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Create callback to stop training process after getting >90% accuracy
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>0.90 and logs.get('val_accuracy')>0.90):
            self.model.stop_training=True
            print("\nThe accuracy of training and validation data has reached > 90%")

callbacks = myCallback()

In [24]:
# Train the model
num_epochs = 5

history = model.fit(padding_train, y_train, epochs=num_epochs,
                    validation_data=(padding_valid, y_valid),
                    verbose=1,
                    callbacks=[callbacks]
                   )

Epoch 1/5
1281/1281 [==============================] - 1547s 1s/step - loss: 0.4775 - accuracy: 0.7981 - val_loss: 0.5226 - val_accuracy: 0.7166
Epoch 2/5
1281/1281 [==============================] - ETA: 0s - loss: 0.2605 - accuracy: 0.9033
The accuracy of training and validation data has reached > 90%
1281/1281 [==============================] - 1524s 1s/step - loss: 0.2605 - accuracy: 0.9033 - val_loss: 0.1738 - val_accuracy: 0.9407


In [26]:
model.save('SNN_fake_news.h5')

In [ ]:
# Use the trained model to predict validation data
y_pred = model.predict(padding_valid)
y_pred = (y_pred > 0.5).astype(int)

# Show classification metrics
model_report = classification_report(y_valid, y_pred, target_names=['Fake', 'True'])
print(model_report)